In [ ]:
#Não sei se funciona ainda

import pandas as pd
import re #biblioteca de expressões regulares

# Carregar o CSV
df = pd.read_csv('tabela_saci.csv')

def parse_schedule(schedule_str): #função de tradução de analisar sintaticamente uma tabela
    """
    Transforma '35M23' em uma lista de tuplas de tempo.
    Ex: '35M23' -> [('3', 'M', '2'), ('3', 'M', '3'), ('5', 'M', '2'), ('5', 'M', '3')]
    """
    if pd.isna(schedule_str) or schedule_str == 'A Definir':   
        return []
    
    # Regex para separar dias, turno e horários
    match = re.match(r'([2-7]+)([MTN])([1-6]+)', schedule_str)
    if not match:
        return [] # Formato desconhecido ou complexo
    
    days, shift, hours = match.groups()
    slots = []
    for day in days:
        for hour in hours:
            slots.append((day, shift, hour))
    return slots

# 1. Preparar as Disciplinas (Variáveis)
disciplinas = []
for idx, row in df.iterrows():
    # Estamos assumindo que o horário é um requisito FIXO da disciplina 
    # e o objetivo é encontrar uma SALA que não dê conflito.
    # (Se o objetivo for mudar o horário também, a complexidade aumenta muito).
    disciplina = {
        'id': idx,
        'nome': row['Disciplina'],
        'prof': row['Professor(a)'],
        'slots': parse_schedule(row['Horário']),
        'sala_original': row['Local'] # Apenas para comparação depois
    }
    disciplinas.append(disciplina)

# 2. Preparar as Salas (Domínio)
# Vamos pegar todas as salas únicas que aparecem no arquivo
salas_unicas = df['Local'].unique()
salas = [s for s in salas_unicas if pd.notna(s) and s != 'A Definir']

print(f"Total de Disciplinas para alocar: {len(disciplinas)}")
print(f"Total de Salas disponíveis: {len(salas)}")
print("Exemplo de disciplina processada:", disciplinas[0])